<a href="https://colab.research.google.com/github/rashida048/Tensorflow_Advance_Techniques/blob/main/intro_to_tf_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorflow==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.13.0rc0, 2.13.0rc1)
ERROR: No matching distribution found for tensorflow==2.4.0


In [4]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/tfdata-intro/tfdata-intro.zip
!unzip -qq tfdata-intro.zip
%cd tfdata-intro

--2023-06-01 03:28:59--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/tfdata-intro/tfdata-intro.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.218.153.129, 3.5.83.150, 52.92.129.234, ...
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.218.153.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 586970111 (560M) [binary/octet-stream]
Saving to: ‘tfdata-intro.zip’

tfdata-intro.zip    100%[===================>] 559.78M  27.5MB/s    in 18s     

2023-06-01 03:29:18 (31.0 MB/s) - ‘tfdata-intro.zip’ saved [586970111/586970111]

/content/tfdata-intro


In [5]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar100
from tensorflow.data import AUTOTUNE
from imutils import paths
import tensorflow as tf
import numpy as np
import time
import os

In [6]:
def benchmark(datasetGen, numSteps):
  start = time.time() 

  for i in range(0, numSteps):
    (images, labels) = next(datasetGen)

  end = time.time()

  return (end - start)

In [7]:
BS = 64 
NUM_STEPS = 5000 

((trainX, trainY), (testX, testY)) = cifar100.load_data()

169001437/169001437 [==============================] - 3s 0us/step


In [11]:
imageGen = ImageDataGenerator()
dataGen = imageGen.flow(
    x=trainX, y = trainY,
    batch_size = BS, shuffle=True 
)

In [12]:
dataset = tf.data.Dataset.from_tensor_slices((trainX, trainY))

dataset=(dataset.shuffle(1024)
.cache()
.repeat()
.batch(BS)
.prefetch(AUTOTUNE)
)

In [13]:
totalTime = benchmark(dataGen, NUM_STEPS)
print("[INFO] ImageDataGenerator generated {} images in " \
      " {:.2f} seconds...".format(
          BS * NUM_STEPS, totalTime
      ))

[INFO] ImageDataGenerator generated 320000 images in  10.33 seconds...


In [14]:
datasetGen = iter(dataset)
totalTime = benchmark(datasetGen, NUM_STEPS)
print("[INFO] tf.data generated {} images in {:.2f} seconds...".format(
	BS * NUM_STEPS, totalTime))

[INFO] tf.data generated 320000 images in 1.97 seconds...


In [15]:
def load_images(imagePath):
	# read the image from disk, decode it, resize it, and scale the
	# pixels intensities to the range [0, 1]
	image = tf.io.read_file(imagePath)
	image = tf.image.decode_png(image, channels=3)
	image = tf.image.resize(image, (96, 96)) / 255.0

	# grab the label and encode it
	label = tf.strings.split(imagePath, os.path.sep)[-2]
	oneHot = label == classNames
	encodedLabel = tf.argmax(oneHot)

	# return the image and the integer encoded label
	return (image, encodedLabel)

In [16]:
args = {
    "dataset": "fruits"
}

In [17]:
BS = 64
NUM_STEPS = 1000

imagePaths = list(paths.list_images(args["dataset"]))
classNames = np.array(sorted(os.listdir(args["dataset"])))

In [18]:
dataset = tf.data.Dataset.from_tensor_slices(imagePaths)
dataset = (dataset.shuffle(1024)
.map(load_images, num_parallel_calls = AUTOTUNE)
           .cache()
           .repeat()
           .batch(BS)
           .prefetch(AUTOTUNE)
           )

In [19]:
# create a standard image generator object
print("[INFO] creating a ImageDataGenerator object...")
imageGen = ImageDataGenerator(rescale=1.0/255)
dataGen = imageGen.flow_from_directory(
	args["dataset"],
	target_size=(96, 96),
	batch_size=BS,
	class_mode="categorical",
	color_mode="rgb")

[INFO] creating a ImageDataGenerator object...
Found 6688 images belonging to 7 classes.


In [20]:
totalTime = benchmark(dataGen, NUM_STEPS)
print("[INFO] ImageDataGenerator generated {} images in " \
	  " {:.2f} seconds...".format(
	BS * NUM_STEPS, totalTime))

[INFO] ImageDataGenerator generated 64000 images in  380.54 seconds...


In [21]:
datasetGen = iter(dataset)
totalTime = benchmark(datasetGen, NUM_STEPS)
print("[INFO] tf.data generated {} images in {:.2f} seconds...".format(
	BS * NUM_STEPS, totalTime))

[INFO] tf.data generated 64000 images in 19.50 seconds...
